In [ ]:
#!conda install -y -c conda-forge rdkit
#!conda install -c conda-forge py3dmol

In [ ]:
import re
import numpy
import pandas
from matplotlib import pyplot
import tkinter
import tkinter.filedialog
import tkinter.messagebox
import rdkit
import py3Dmol
#!git clone https://github.com/jensengroup/xyz2mol
from xyz2mol import xyz2mol

In [ ]:
# periodic table
periodictable = {"1": "H", "6": "C", "7": "N", "8": "O", "9": "F", "15": "P", "16": "S", "17": "Cl"}

In [ ]:
# file open
root = tkinter.Tk()
root.withdraw()
root.attributes('-topmost', True)
root.lift()
root.focus_force()
filename = tkinter.filedialog.askopenfilename(filetypes=[("out", "*.out")])
f = open(filename, "r")
data = f.readlines()
f.close()
print(filename)

In [ ]:
# energies
buff = [s for s in data if "SCF Done:  E(" in s]
buff = "".join(buff)
buff = re.findall("-" + r"\d+" + "." + r"\d+", buff)
buff = numpy.array(buff, dtype="float")
pyplot.plot(buff)
pyplot.show()
'''
print("SCF Done:  E =")
print(buff[-1], "Hartree")
print(buff[-1] * 2625.5, "kJ mol^-1")
print("")
'''
buff = [s for s in data if "Sum of electronic and zero-point Energies=" in s]
buff = "".join(buff)
buff = re.findall("-" + r"\d+" + "." + r"\d+", buff)
buff = numpy.array(buff, dtype="float")[0]
print("Sum of electronic and zero-point Energies =")
print(buff, "Hartree")
print(buff * 2625.5, "kJ mol^-1")

In [ ]:
# optimized structure
start = data.index("    -- Stationary point found.\n")
buff = data[start:]
stop = buff.index("                    Distance matrix (angstroms):\n")
buff = buff[:stop]
start = (buff.index("                          Input orientation:                          \n") + 5)
buff = buff[start:]
stop = buff.index(" ---------------------------------------------------------------------\n")
buff = buff[:stop]
xyz = pandas.DataFrame()
for i in range(len(buff)): xyz[i] = re.findall(r"([+-]?[0-9]+\.?[0-9]*)", buff[i])
xyz = xyz.drop(xyz.index[[0, 2]])
xyz.iloc[0] = xyz.iloc[0].replace(periodictable)
xyz.T.to_csv(filename[:-4] + ".xyz", header=False, index=False, sep=" ")
f = open(filename[:-4] + ".xyz", "r")
buff = f.read()
f.close()
f = open(filename[:-4] + ".xyz", "w")
f.write(str(len(xyz.T)) + "\ncharge=0=\n" + buff)
f.close()
print(filename[:-4] + ".xyz")
xyz.T

In [ ]:
# IR spectrum
buff = [s for s in data if "Frequencies" in s]
buff = "".join(buff)
x = re.findall(r"([+-]?[0-9]+\.?[0-9]*)", buff)
x = numpy.round(numpy.array(x, dtype="float") * 0.9613, 2)
buff = [s for s in data if "IR Inten" in s]
buff = "".join(buff)
y = re.findall(r"\d+" + "." + r"\d+", buff)
y = numpy.round(numpy.array(y, dtype="float"), 2)
pyplot.xlim(4000, 0)
pyplot.bar(x, y, width=30)
pyplot.show()
spec = pandas.Series(y, index=x)
pandas.set_option('display.max_rows', None)
spec
#spec.to_csv(filename[:-4]+".csv", header=False)

In [ ]:
# IR spectrum
def g(px, py):
    w = 15
    gauss = py * numpy.exp(-4 * numpy.log(2) * (px - x) ** 2 / w ** 2)
    return gauss
x = numpy.arange(0, 4001, 1)
y = x * 0
for i in range(len(spec)): y = y + g(spec.index[i], spec.values[i])
pyplot.xlim(4000, 0)
pyplot.plot(x, y)
pyplot.show()
#pandas.Series(y, index=x).to_csv(filename[:-4]+".csv", header=False)

In [ ]:
# 3D
atoms, charge, xyz_coordinates = xyz2mol.read_xyz_file(filename[:-4] + ".xyz")
mols = xyz2mol.xyz2mol(atoms, xyz_coordinates, charge=charge)
view = py3Dmol.view()
view.addModel(rdkit.Chem.MolToMolBlock(mols[0]))
view.setStyle({'stick': {}})
view.show()